# Freqeuntism and Bayesianism: A python-driven Primer
### Jake VanderPlas, edited by Crane Huang
Reference: http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/

## 1. When they are the same: Photon Flux Measurements

Imagine that we point a telescope to the sky, and observe the light coming from a single star. Assuming that the star's true photo flux is a fixed value F, and a series of N measurements are perfromed, where the i_th measurement reports the observed flux F_i and error e_i. 

The question is: given this set of measurements D={F,e}, what is the best esimate of the true flux F?

In [1]:
import numpy as np

In [2]:
e=np.random.normal(30,3,50)# generate 50 toy data points: error term.
F=np.random.normal(1000,e) # generate 50 toy data points: flux F. (true flux F is 1000)

### - Frequentist Approach: 
Maximum Likelihood Estimation (MLE): The result is a simple weighted mean of the observed values.

In [3]:
w=1./e**2
F_hat=np.sum(w*F)/np.sum(w)
sigma_F=w.sum() ** -.5
F_hat

1001.6444481772371

### - Bayesian Approach: 

P(F|D) = P(D|F)P(F)/P(D) 

#### Equivalent to Frequentist Approach with a flat prior

## 2. When they differ: Bayes' Billiards Game
Alice and Bob enter a room to play a game. Carol rolls a ball down the table and marks where it lands. Then Carol begins rolling new balls down the table. If the ball lands to the left of the mark, Alice gets a point; if it lands to the right of the mark, Bob gets a point. The first person to reach 6 points wins the game.

In a particular game, after 8 rolls, Alice has 5 points and Bob has 3 points. What is the probability that Bob will 6 points and win the game?

### - Frequentist Approach: 
The MLE of P(B)=3/8, and the only senario that Bob can win is to get 3 points in a row.

In [4]:
p_b_freq=(3/8)**3
print("Naive Frequentist Probability of Bob winning: %.2f" %(p_b_freq))

Naive Frequentist Probability of Bob winning: 0.05


### - A Bayesian Approach:
#### A beta-binomial distribution, with theta=P(B), prior p(theta|a,b)=Beta(theta|a,b), posterior P(theta|D)~Beta(theta| N1+a, N0+b), where N1=points for Bob, and N0=points for Alice.
#### Given a uninformative prior beta (1,1) and likelihood D=(3,5), the posterior predictive prob is Beta(1+6,1+5)/Beta(1+3,1+5); 
Proof: with prior Beta(1,1) and Likelihood (3,5), we have a posterior distribution of Beta(1+3, 1+5) and can be used as a new prior.
For Bob to win, he needs to win 3 games in a row and the data will be D=(3,0), thus P=Beta(3+1+3, 1+5)/Beta(1+3, 1+5), with the posterior distribution Beta(1+3, 1+5) as the new prior. 
Detailed derivation can be found on P80 in Machine Learning: a probablistic perspective (Kevin Murphy) or on the website provided above.

In [5]:
from scipy.special import beta

In [6]:
p_b_bay=beta(1+6,1+5)/beta(1+3,1+5)
print("Bayesian posterior probabilty of Bob winning: %.2f" %(p_b_bay))

Bayesian posterior probabilty of Bob winning: 0.09


### - Monte Carlo Simulation

In [9]:
p = np.random.random(100000) #play 100000 games with randomly-drawn p
rolls=np.random.random((11,len(p))) #each game needs at most 11 rolls for one player to reach 6 wins

A_count=np.cumsum(rolls<p,0)  #count Alice's total wins at each roll
B_count=np.cumsum(rolls>=p,0) #count Bob's total wins at each roll
good_games=B_count[7]==3      #determin number of games have (A wins, B wins)=(5,3)
A_count=A_count[:,good_games] #keep only suitable games
B_count=B_count[:,good_games]

B_won=np.sum(B_count[10]==6) #determin which games Bob won

mc_prob=B_won/good_games.sum()
print("Monte Carlo probability of Bob winning: %.2f" %(mc_prob))

Monte Carlo probability of Bob winning: 0.09
